## Null value check for table [bronze_daily]

In [0]:
from pyspark.sql.functions import col, sum

df = spark.read.table("kenworkspace.tw_stocks_db.bronze_daily")

# 假設 df 是你的 DataFrame
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()


##Null value check for table [bronze_monthly]

In [0]:
df = spark.read.table("kenworkspace.tw_stocks_db.bronze_monthly")

# 假設 df 是你的 DataFrame
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

## Feature engineering for table [bronze_monthly] --> [silver_mvrh_monthly]
### 1. New column: monthly volume record high [MVRH]
### 2. New column: Distance (%) from last MVRH [from_MVRH_percent]
### 3. Get rid of the symbol column

In [0]:
%sql
CREATE OR REPLACE TABLE kenworkspace.tw_stocks_db.silver_mvrh_monthly AS
  WITH base AS (
    SELECT
      `date`,
      `open`,
      `high`,
      `low`,
      `close`,
      `volume`,
      MAX(`volume`) OVER (ORDER BY `date`) AS max_volume_so_far
    FROM kenworkspace.tw_stocks_db.bronze_monthly
  ),
  mvrh_flagged AS (
    SELECT *,
      volume = max_volume_so_far AS mvrh
    FROM base
  ),
  add_previous_mvrh AS (
    SELECT *,
      -- 找出每一筆資料之前的最近一次 MVRH 的 volume（排除自己）
      LAG(
        CASE WHEN mvrh THEN volume ELSE NULL END
      ) IGNORE NULLS OVER (ORDER BY `date`) AS last_mvrh_volume
    FROM mvrh_flagged
  ),
  final AS (
    SELECT *,
      -- 計算與前一次 MVRH 的百分比差距
      CASE 
        WHEN last_mvrh_volume IS NOT NULL THEN 
          ROUND((volume - last_mvrh_volume) / last_mvrh_volume * 100, 2)
        ELSE NULL 
      END AS from_mvrh_percent
    FROM add_previous_mvrh
  )
  SELECT * FROM final;

##Feature engineering for table [bronze_daily] --> [silver_daily]
###1. Get rid of the symbol column 
###2. New column: 60 days future highest return [60fhr_percent]
###3. New column: 60 days future lowest return [60flr_percent]

In [0]:
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import StructType, StructField, DoubleType
import pandas as pd

# 讀取數據，按日期排序
df = spark.table("kenworkspace.tw_stocks_db.bronze_daily").orderBy("date")

# 定義 pandas_udf，輸入是多個 pd.Series，輸出是 DataFrame
@pandas_udf(returnType=StructType([
    StructField("60fhr_percent", DoubleType(), True),
    StructField("60flr_percent", DoubleType(), True)
]))
def calculate_future_returns(closes: pd.Series, highs: pd.Series, lows: pd.Series) -> pd.DataFrame:
    results = []

    for i in range(len(closes)):
        start_idx = i + 1
        end_idx = min(i + 61, len(closes))
        
        # 判斷未來是否有滿60個交易日資料
        if (end_idx - start_idx) < 60:
            # 不足60天，回傳 None
            results.append({"60fhr_percent": None, "60flr_percent": None})
            continue
        
        future_highs = highs.iloc[start_idx:end_idx]
        future_lows = lows.iloc[start_idx:end_idx]
        current_close = closes.iloc[i]

        max_high = future_highs.max()
        max_high_pos = future_highs.idxmax()  # 全局 index
        max_high_relative_pos = max_high_pos - start_idx

        fhr_percent = round((max_high - current_close) / current_close * 100, 2)

        before_max_lows = future_lows.iloc[:max_high_relative_pos + 1]
        min_low = before_max_lows.min()
        flr_percent = round((min_low - current_close) / current_close * 100, 2)

        results.append({
            "60fhr_percent": fhr_percent,
            "60flr_percent": flr_percent
        })

    return pd.DataFrame(results)

# 用 select 傳入多欄位給 pandas_udf，並用 alias 指定欄位名稱
df_with_return = df.select(
    "*",  # 保留所有欄位
    calculate_future_returns(
        col("close"),
        col("high"),
        col("low")
    ).alias("future_returns")
)

# 拆解 struct 欄位成獨立欄位
result_df = df_with_return.select(
    "date", "open", "high", "low", "close", "volume",
    col("future_returns.60fhr_percent").alias("60fhr_percent"),
    col("future_returns.60flr_percent").alias("60flr_percent")
)

# 寫入 Delta 表格
result_df.write.format("delta").mode("overwrite").saveAsTable("kenworkspace.tw_stocks_db.silver_daily")

print("Successfully created silver_daily table!")


In [0]:
%sql
USE CATALOG kenworkspace;

SELECT *
FROM tw_stocks_db.silver_daily
ORDER BY date DESC
LIMIT 100;